# Synthetix V3


In [ ]:
import os
import time
from synthetix import Synthetix
from synthetix.utils import wei_to_ether, ether_to_wei, format_wei, format_ether
from dotenv import load_dotenv

load_dotenv()

In [ ]:
snx = Synthetix(
    provider_rpc=os.getenv("LOCAL_RPC"),
    private_key=os.getenv("PRIVATE_KEY"),
    network_id=84532,
    is_fork=True,
    cannon_config={
        "package": "synthetix-omnibus",
        "version": "33",
        "preset": "andromeda",
    },
)

### Setup


In [ ]:
# create some needed contracts
weth = snx.contracts["WETH"]["contract"]

# USDC token
usdc_package = snx.contracts["packages"]["usdc_mock_collateral"]["MintableToken"]
usdc = snx.web3.eth.contract(address=usdc_package["address"], abi=usdc_package["abi"])

# aUSDC token
ausdc_package = snx.contracts["packages"]["ausdc_token_mock"]["MintableToken"]
ausdc = snx.web3.eth.contract(
    address=ausdc_package["address"], abi=ausdc_package["abi"]
)

# stataUSDC oracle
statausdc_oracle_package = snx.contracts["packages"]["erc_4626_to_assets_ratio_oracle"][
    "ERC4626ToAssetsRatioOracle"
]
statausdc_oracle = snx.web3.eth.contract(
    address=statausdc_oracle_package["address"],
    abi=statausdc_oracle_package["abi"],
)

# stataUSDC vault
vault_address = statausdc_oracle.functions.VAULT_ADDRESS().call()
vault = snx.web3.eth.contract(
    address=vault_address,
    abi=snx.contracts["ERC4626"]["abi"],
)

In [ ]:
def status():
    """A function printing some useful information about the connected account"""
    eth_balance = snx.get_eth_balance()
    susd_balance = snx.spot.get_balance(market_name="sUSD")
    susdc_balance = snx.spot.get_balance(market_name="sUSDC")

    usdc_balance = usdc.functions.balanceOf(snx.address).call()
    usdc_balance = format_wei(usdc_balance, 6)

    ausdc_balance = ausdc.functions.balanceOf(snx.address).call()
    ausdc_balance = format_wei(ausdc_balance, 6)

    vault_balance = vault.functions.balanceOf(snx.address).call()
    vault_balance = format_wei(vault_balance, 6)

    print(
        f"""
    Perps accounts: {snx.perps.account_ids}
    Core accounts: {snx.core.account_ids}
        
    ETH: {eth_balance['eth']}
    WETH ({weth.address}): {eth_balance['weth']}
    USDC ({usdc.address}): {usdc_balance}
    sUSDC: {susdc_balance}
    sUSD: {susd_balance}
    aUSDC: {ausdc_balance}
    StataUSDC: {vault_balance}
    """
    )


status()